# XASH Testing

In [2]:
import pandas as pd
import pyarrow as pa

import math
from collections import Counter
import numpy as np




In [ ]:
!pwd

In [ ]:
def XASH(token: str, hash_size: int = 128) -> int:
    """Computes XASH for given token.

    Parameters
    ----------
    token : str
        Token.

    hash_size : int
        Number of bits.

    Returns
    -------
    int
        XASH value.
    """
    number_of_ones = 5
    char = [' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
            'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
    segment_size_dict = {64: 1, 128: 3, 256: 6, 512: 13}
    segment_size = segment_size_dict[hash_size]
    length_bit_start = 37 * segment_size
    result = 0
    cnt_dict = Counter(token)
    selected_chars = [y[0] for y in sorted(cnt_dict.items(), key=lambda x: (x[1], x[0]), reverse=False)[:number_of_ones]]
    for c in selected_chars:
        if c not in char:
            continue
        indices = [i for i, ltr in enumerate(token) if ltr == c]
        mean_index = np.mean(indices)
        token_size = len(token)
        for i in np.arange(segment_size):
            if mean_index <= ((i + 1) * token_size / segment_size):
                location = char.index(c) * segment_size + i
                break
        result = result | int(math.pow(2, location))

    # rotation
    n = int(result)
    d = int((length_bit_start * (len(token) % (hash_size - length_bit_start))) / (
                hash_size - length_bit_start))
    int_bits = int(length_bit_start)
    x = n << d
    y = n >> (int_bits - d)
    r = int(math.pow(2, int_bits))
    result = int((x | y) % r)

    result = int(result) | int(math.pow(2, len(token) % (hash_size - length_bit_start)) * math.pow(2, length_bit_start))

    return result

In [ ]:
# Load the sample dataframe list

df_sample_list = '/tank/local/suhail/data/relic-datalake/gittables/samples/5000_sample.txt'

with open(df_sample_list, 'r') as fp:
    file_list = [eval(x) for x in fp.readlines()]
            
sample_df = pd.read_parquet(file_list[0])


In [ ]:
sample_df

In [ ]:
sample_row = sample_df.iloc[0]
sample_row.index

In [ ]:
list(sample_row.iteritems())

In [ ]:
from collections import defaultdict


def generate_hash_superkey(row, hash_function=XASH, hash_size=128):
    superkey = 0
    value_list = []
    for colname, val in row.iteritems():
        value = str(val)
        superkey = superkey | hash_function(value, hash_size)
        value_list.append((colname, value))
    return superkey, value_list

def index_dataframe(df):
    for idx, row in df.iterrows():
        superkey, value_list = generate_hash_superkey(row)
        yield idx, superkey, value_list
        
        
def generate_posting_list(df, label='label'):
    posting_list = defaultdict(list)
    for idx, superkey, value_list in index_dataframe(df):
        for colname, value in value_list:
            posting_list[value].append((label, idx, colname, superkey))
    return posting_list



In [ ]:
generate_hash_superkey(sample_row, XASH, 128)

In [ ]:
generate_posting_list(sample_df)

In [ ]:
sample_row

# Arrow Server Testing

In [2]:
import sys
sys.path.append('../')

from src.arrow_client import *

import pandas as pd
import glob
import os

In [3]:
client = FlightSketchClient()

In [ ]:
sample_df

In [ ]:
client.constuct_index_df(pa.table(sample_df), 'sample')

In [5]:
client.serialize_sketches()

# Querying Prototyping


In [ ]:
import pickle

server_serialized_file = "MATE/src/sketchset.ser"

with open(server_serialized_file, 'rb') as fp:
    posting_list, table_pl = pickle.load(fp)

In [ ]:
!pwd

In [ ]:
table_pl.keys()

In [ ]:
# Load query dataframe
query_output_dir='/tank/local/suhail/data/relic-datalake/gittables/outputs/100_queries/artifacts/'
query_ops_file='/tank/local/suhail/data/relic-datalake/gittables/outputs/100_queries/operations.parquet'

query_df = pd.read_parquet(query_ops_file)
query_df

In [ ]:
gb_queries = query_df.loc[(query_df.operation == 'groupby') & (query_df.colset_size == 2)]
gb_queries

In [ ]:
sample_query_df = pd.read_parquet(gb_queries.iloc[0]['dst_label'])
query_cols = gb_queries.iloc[0]['args']['colset']
query_cols


In [ ]:
q_df = sample_query_df[query_cols]
q_df

In [ ]:
def perform_ICS(query_df):
    all_cardinalities = query_df.apply(lambda x: x.nunique())
    return all_cardinalities.idxmin()


def initial_pl_filter(query_df, posting_list):
    init_column = perform_ICS(query_df)
    init_values = set(query_df[init_column])
    
    


perform_ICS(q_df)

In [ ]:
from collections import Counter


def generate_hash_superkey(row, hash_function=XASH, hash_size=128):
    superkey = 0
    value_list = []
    for colname, val in row.iteritems():
        value = str(val)
        superkey = superkey | hash_function(value, hash_size)
        value_list.append((colname, value))
    return superkey, value_list

def perform_initial_table_filtering(q_df, posting_list):
    init_column = perform_ICS(q_df)
    init_values = set(q_df[init_column].values.astype(str))

    initial_table_list = defaultdict(list)
    for value in init_values:
        for tableid, rowid, colid, superkey in posting_list[value]:
            initial_table_list[tableid].append((value, rowid, colid, superkey))
            
    return sorted(initial_table_list.items(), key=lambda x: len(x[1]), reverse=True)


def perform_row_filtering(q_df, posting_list, table_pl, initial_table_list, superkeydf, init_col_label):
    ranked_table_list = []
    for tableid, table_pls in initial_table_list:
        # TODO: Early stopping condition
        matching_rows = []
        for value, rowid, colid, superkey in table_pls:
            # Get list of unqiue superkeys for the value in the query table
            # TODO: Second early stopping condition based on num_rows checked
            init_row_match = superkeydf.loc[superkeydf[init_col_label] == value]
            for idx, sk_row in init_row_match.iterrows():
                if sk_row['superkey'] | superkey == superkey:
                     matching_rows.append((tableid, rowid, colid, value))
            
        ranked_table_list.append((tableid, matching_rows))
    
    return sorted(ranked_table_list, key=lambda x: len(x[1]), reverse=True)
    

    
def get_query_superkey(q_df, hash_function=XASH, hash_size=128):
    q_sk_df = q_df.copy()
    q_sk_df['superkey'] = q_sk_df.apply(lambda x: generate_hash_superkey(x, hash_function, hash_size)[0], axis=1)
    return q_sk_df


def process_gb_query(gb_query, posting_list, table_pl):
    q_df = pd.read_parquet(gb_query['dst_label'])[gb_query['args']['colset']]
    superkeydf = get_query_superkey(q_df)
    init_table_list = perform_initial_table_filtering(q_df, posting_list)
    final_ranked_list = perform_row_filtering(q_df, posting_list, table_pl, init_table_list, superkeydf, perform_ICS(q_df))
    return [x[0] for x in final_ranked_list]


gb_results = gb_queries.copy()

#gb_results['results'] = gb_results.apply(lambda x: process_gb_query(x, posting_list, table_pl), axis=1)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
# Test One Query
gb_query = gb_results.iloc[2]
q_df = pd.read_parquet(gb_query['dst_label'])[gb_query['args']['colset']]
superkeydf = get_query_superkey(q_df)
init_table_list = perform_initial_table_filtering(q_df, posting_list)
#final_ranked_list = perform_row_filtering(q_df, posting_list, table_pl, init_table_list, superkeydf, perform_ICS(q_df))
init_table_list

In [ ]:
final_ranked_list = perform_row_filtering(q_df, posting_list, table_pl, init_table_list, superkeydf, perform_ICS(q_df))

In [ ]:
len(init_table_list)

In [ ]:
q_df#.describe()

In [ ]:
gb_queries.iloc[2]

In [ ]:
gb

In [ ]:
gb_results.to_parquet('gb_results.parquet')

In [ ]:
gb_results


In [ ]:
gb_results['src_filename'] = gb_results['src_labels'].apply(lambda x: os.path.basename(x[0]))

In [ ]:
gb_results

In [10]:
def verify_groupby_result_in_k(row, k=10):
    correct_src = row['src_filename']
    found_correct_src = -1
    ranked_results = 0
    total_results = 0
    
    try:
        total_results = len(row['results'])
        
        for ix, rank_item in enumerate(row['results']):
            if rank_item == correct_src and found_correct_src == -1:
                found_correct_src = ranked_results
            ranked_results += 1
    except Exception as e:
        #print(e)
        pass
            
    return pd.Series([found_correct_src, ranked_results, total_results])

def verify_pivot_result_in_k(row, k=10):
    correct_src = row['src_filename']
    found_correct_src = -1
    ranked_results = 0
    total_results = len(row['results'])
    
    for ix, rank_item in enumerate(row['results']):
        if rank_item == correct_src  and found_correct_src == -1:
            found_correct_src = ranked_results
        ranked_results += 1
            
    return pd.Series([found_correct_src, ranked_results, total_results])


def compute_recall_rate_at_k(rank_array: pd.Series, k: int = 10):
    correct_values = rank_array.loc[rank_array != -1]
    return sum(correct_values < k) / len(rank_array)

def get_query_statistics(result_df, col='total_results', rename_col='total_results'):
    stat_df = pd.DataFrame(result_df[col].describe()) 
    stat_df.rename(columns={col: rename_col}, inplace=True)
    return stat_df.T

def generate_top_k_df(result_df, result_col, k_range=[1,5,10], op_label='operation'):
    result_dict = {}
    for k in [1,5,10]:
        result_dict[f"k={k}"] = compute_recall_rate_at_k(result_df[result_col], k=k)
    return pd.DataFrame(result_dict, index=[op_label])




In [5]:
!pwd


/tank/local/suhail/MATE/notebooks


In [7]:
gb_results = pd.read_parquet('../src/gb_results_2col_MATE.parquet')
gb_results['src_filename'] = gb_results['src_labels'].apply(lambda x: os.path.basename(x[0]))
gb_results[['correct_src_rank', 'ranked_results', 'total_results']] = gb_results.apply(verify_groupby_result_in_k, axis=1)
groupby_qstats = get_query_statistics(gb_results, col='total_results', rename_col='groupby (2-col)')
groupby_acc = generate_top_k_df(gb_results, result_col='correct_src_rank', op_label='groupby (2-col)')
groupby_nums = pd.concat([groupby_qstats, groupby_acc], axis=1, keys=['Total Query Results', 'Ranking Rate @ k'])
groupby_nums

Total Query Results                                     \
                              count     mean          std  min     25%   
groupby (2-col)               100.0  1870.35  1442.200349  1.0  137.75   

                                        Ranking Rate @ k              
                    50%     75%     max              k=1   k=5  k=10  
groupby (2-col)  1809.0  3368.0  3499.0             0.23  0.26  0.29

In [11]:
pivot_results = pd.read_parquet('../src/pivot_results_2col_MATE.parquet')
pivot_results['src_filename'] = pivot_results['src_labels'].apply(lambda x: os.path.basename(x[0]))
pivot_results[['correct_src_rank', 'ranked_results', 'total_results']] = pivot_results.apply(verify_pivot_result_in_k, axis=1)
pivot_qstats = get_query_statistics(pivot_results, col='total_results', rename_col='pivot (2-col)')
pivot_acc = generate_top_k_df(pivot_results, result_col='correct_src_rank', op_label='pivot (2-col)')
pivot_nums = pd.concat([pivot_qstats, pivot_acc], axis=1, keys=['Total Query Results', 'Ranking Rate @ k'])
pivot_nums

Total Query Results                                         \
                            count   mean         std  min  25%  50%  75%   
pivot (2-col)               100.0  24.71  179.326028  1.0  1.0  1.0  8.0   

                      Ranking Rate @ k              
                  max              k=1   k=5  k=10  
pivot (2-col)  1795.0              0.8  0.95  0.95

In [9]:
pivot_results

,src_labels,operation,args,dst_label,colset_size,results,src_filename,correct_src_rank,ranked_results,total_results
300,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[J01-2004_11.parquet, W11-2123_aakansha_108.pa...",J01-2004_11.parquet,-1,5,5
301,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[dmsynth.dll.csv__3705671084__.parquet],dmsynth.dll.csv__3705671084__.parquet,-1,1,1
302,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[gameId=0041900235.parquet],gameId=0041900235.parquet,-1,1,1
303,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ConfigureExpandedStorage.dll.csv__2376278011_...,ConfigureExpandedStorage.dll.csv__2376278011__...,-1,1,1
304,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[vm3dum64.dll.csv__17705870__.parquet],vm3dum64.dll.csv__17705870__.parquet,-1,1,1
...,...,...,...,...,...,...,...,...,...,...
395,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[NEON.D02.SCBI.DP1.10072.001.mam_perplotnight....,NEON.D02.SCBI.DP1.10072.001.mam_perplotnight.p...,-1,1,1
396,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[tweets-@h3llatrash.parquet],tweets-@h3llatrash.parquet,-1,1,1
397,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ncobjapi.dll.csv__3645063681__.parquet],ncobjapi.dll.csv__3645063681__.parquet,-1,1,1
398,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[Configuration3_window15_split12_typetest_proj...,Configuration3_window15_split9_typetest_projec...,-1,12,12


In [35]:
# Join Results


def verify_join_column_in_original(ranking_keys, src_labels):
    left_colset = set(x.replace('__LEFT', '') for x in ranking_keys if '__LEFT' in x)
    right_colset = set(x.replace('__RIGHT', '') for x in ranking_keys if '__RIGHT' in x)
                
    #print(left_colset, right_colset)
    join_src_mapping = {}
    
    for label in src_labels:
        src_df = pd.read_parquet(label)
        if set(src_df.columns).issuperset(left_colset):
            join_src_mapping['left'] = os.path.basename(label)
        if set(src_df.columns).issuperset(right_colset):
            join_src_mapping['right'] = os.path.basename(label)
    return join_src_mapping['left'], join_src_mapping['right']


def verify_join_columns(row):
    dst_cols = set(pd.read_parquet(row['dst_label']).columns)
    return verify_join_column_in_original(dst_cols, row['src_labels'])


def verify_join_result_in_k(row, k=10, side=0):
    correct_src = verify_join_columns(row)[side]
    found_correct_src = -1
    ranked_results = 0
    total_results = 0
    
    try:
        total_results = len(row['results'][side])
        
        for ix, rank_item in enumerate(row['results'][side]):
            if rank_item == correct_src and found_correct_src == -1:
                found_correct_src = ranked_results
            ranked_results += 1
    except Exception as e:
        #print(e)
        pass
            
    return pd.Series([found_correct_src, ranked_results, total_results])
    

join_results = pd.read_parquet('../src/join_results_2col_MATE.parquet')
join_results['src_filename'] = join_results['src_labels'].apply(lambda x: os.path.basename(x[0]))
join_results[['correct_src_rank', 'ranked_results', 'total_results']] = join_results.apply(verify_join_result_in_k, axis=1)
join_qstats = get_query_statistics(join_results, col='total_results', rename_col='join (2-col)')
join_acc = generate_top_k_df(join_results, result_col='correct_src_rank', op_label='join (2-col)')
join_nums = pd.concat([join_qstats, join_acc], axis=1, keys=['Total Query Results', 'Ranking Rate @ k'])
join_nums


Total Query Results                                         \
                           count     mean         std  min   25%    50%   
join (2-col)               100.0  1317.09  1459.74389  1.0  4.75  691.5   

                             Ranking Rate @ k              
                 75%     max              k=1   k=5  k=10  
join (2-col)  3011.0  3488.0             0.35  0.38  0.44

In [3]:
join_results = pd.read_parquet('../src/join_results_2col_MATE.parquet')
join_results['src_filename'] = join_results['src_labels'].apply(lambda x: os.path.basename(x[0]))
join_results[['correct_src_rank', 'ranked_results', 'total_results']] = join_results.apply(verify_join_result_in_k, axis=1, side=1)
join_qstats = get_query_statistics(join_results, col='total_results', rename_col='join (2-col)')
join_acc = generate_top_k_df(join_results, result_col='correct_src_rank', op_label='join (2-col)')
join_nums = pd.concat([join_qstats, join_acc], axis=1, keys=['Total Query Results', 'Ranking Rate @ k'])
join_nums

NameError: name 'verify_join_result_in_k' is not defined

# Analysis of Results (Why are pivots doing so well?)


In [3]:
# Collect the key / group cardinality results
pivot_results = pd.read_parquet('../src/pivot_results_2col_MATE.parquet')

#join_results = pd.read_parquet('../src/join_results_2col_MATE.parquet')
#join_results

pivot_results

,src_labels,operation,args,dst_label,colset_size,results
300,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[J01-2004_11.parquet, W11-2123_aakansha_108.pa..."
301,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[dmsynth.dll.csv__3705671084__.parquet]
302,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[gameId=0041900235.parquet]
303,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ConfigureExpandedStorage.dll.csv__2376278011_...
304,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[vm3dum64.dll.csv__17705870__.parquet]
...,...,...,...,...,...,...
395,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[NEON.D02.SCBI.DP1.10072.001.mam_perplotnight....
396,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[tweets-@h3llatrash.parquet]
397,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ncobjapi.dll.csv__3645063681__.parquet]
398,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[Configuration3_window15_split12_typetest_proj...


In [7]:
def get_query_cardinality(row):
    ''' Assumes df contains ICS first and the other column next'''
    if row['operation'] == 'pivot':
        col_filter = [row['args']['index_col'], row['args']['column_col']]
    elif row['operation'] == 'join':
        col_filter = [row['args']['key_col']]
    elif row['operation'] == 'groupby':
        col_filter = row['args']['colset']
    
    
    df = pd.read_parquet(row['src_labels'][0])[col_filter]
    return len(set(frozenset(x) for x in df.values.tolist()))#, {col: len(set(df[col].values)) for col in df.columns}


pivot_results['query_cardinality'] = pivot_results.apply(get_query_cardinality, axis=1)
pivot_results


,src_labels,operation,args,dst_label,colset_size,results,query_cardinality
300,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[J01-2004_11.parquet, W11-2123_aakansha_108.pa...",17
301,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[dmsynth.dll.csv__3705671084__.parquet],55
302,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[gameId=0041900235.parquet],17
303,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ConfigureExpandedStorage.dll.csv__2376278011_...,96
304,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[vm3dum64.dll.csv__17705870__.parquet],104
...,...,...,...,...,...,...,...
395,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[NEON.D02.SCBI.DP1.10072.001.mam_perplotnight....,99
396,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[tweets-@h3llatrash.parquet],200
397,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ncobjapi.dll.csv__3645063681__.parquet],89
398,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[Configuration3_window15_split12_typetest_proj...,256


In [5]:
pivot_results.query_cardinality.describe()

count    100.000000
mean      68.260000
std       76.883963
min       10.000000
25%       17.000000
50%       28.000000
75%       93.750000
max      313.000000
Name: query_cardinality, dtype: float64

In [42]:
join_results['query_cardinality'] = join_results.apply(get_query_cardinality, axis=1)
join_results

,src_labels,operation,args,dst_label,colset_size,results,query_cardinality
0,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[source_76.parquet, Future_Right2StepUpDown02...",102
1,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[PTN_20190923-191108.parquet], [source_76.par...",26
2,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[Maine_1.parquet, WD-WCAU45488665.parquet, no...",20
3,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[Maine_1.parquet, WD-WCAU45488665.parquet, no...",18
4,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[Delaware.parquet, PL2331LAGM5SHJ.parquet, PL...",15
...,...,...,...,...,...,...,...
95,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[A00-2018_sweta_21.parquet, E03-1005_sweta_10...",16
96,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[Maine_1.parquet, WD-WCAU45488665.parquet, no...",23
97,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[8073.parquet], [Maine_1.parquet, WD-WCAU4548...",26
98,[/tank/local/suhail/data/relic-datalake/gittab...,join,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[[source_76.parquet, Future_Right2StepUpDown02...",16


In [8]:
groupby_results = pd.read_parquet('../src/gb_results_2col_MATE.parquet')
groupby_results['query_cardinality'] = groupby_results.apply(get_query_cardinality, axis=1)

In [9]:
groupby_results.query_cardinality.describe()   

count    100.000000
mean      34.090000
std       46.909346
min       10.000000
25%       16.000000
50%       21.000000
75%       34.250000
max      395.000000
Name: query_cardinality, dtype: float64

In [10]:
pivot_results

,src_labels,operation,args,dst_label,colset_size,results,query_cardinality
300,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,"[J01-2004_11.parquet, W11-2123_aakansha_108.pa...",17
301,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[dmsynth.dll.csv__3705671084__.parquet],55
302,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[gameId=0041900235.parquet],17
303,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ConfigureExpandedStorage.dll.csv__2376278011_...,96
304,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[vm3dum64.dll.csv__17705870__.parquet],104
...,...,...,...,...,...,...,...
395,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[NEON.D02.SCBI.DP1.10072.001.mam_perplotnight....,99
396,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[tweets-@h3llatrash.parquet],200
397,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[ncobjapi.dll.csv__3645063681__.parquet],89
398,[/tank/local/suhail/data/relic-datalake/gittab...,pivot,"{'agg_func': None, 'colset': None, 'column_col...",/tank/local/suhail/data/relic-datalake/gittabl...,NaN,[Configuration3_window15_split12_typetest_proj...,256


In [29]:
from collections import Counter


def count_col_types(df, arg_string):
    counter = Counter()

    for ix, row in df.iterrows():
        counter[row['args'][arg_string]]= counter.get(row['args'][arg_string], 0) + 1
        
    return counter

count_col_types(pivot_results, 'column_col').most_common()

[('Reference Article', 14),
 ('From', 7),
 ('swda_filename', 5),
 ('type', 3),
 ('Title', 2),
 ('Namespace', 2),
 ('Citation Marker', 2),
 ('username', 2),
 ('Discourse Facet', 1),
 ('dmsynth.dll', 1),
 ('game_date', 1),
 ('2041-12-04T20:47:53.000Z', 1),
 ('2019-02-07T17:29:37.000Z', 1),
 ('38cf10ae098d1180db997f6ff2e9fd521c90a4c51d335d9975b1777ebe5548e6', 1),
 ('metric_type', 1),
 ('topic_description', 1),
 ('tua_uuid', 1),
 ('54003fe1c2f797c6ce3ed99f334f343a', 1),
 ('Editor', 1),
 ('b304b0ef47e125f696425bd99096d3e3', 1),
 ('ver', 1),
 ('ee3bd96db188b9703b17b2d6ed392633', 1),
 ('ID', 1),
 ('fms.dll', 1),
 ('Platform', 1),
 ('8c15c6e82cc790c71b09d4fbde7117b1d2a59d49a0604495eb969e9f1de30fc9', 1),
 ('cacls.exe', 1),
 ('week_end_date', 1),
 ('Native_Language', 1),
 ('game_id', 1),
 ('scrape_time', 1),
 ('Type', 1),
 ('FormulaConceptDB', 1),
 ('srds/srds87', 1),
 ('NORESP', 1),
 ('ProductID', 1),
 ('user_id', 1),
 ('clarityColor', 1),
 ('20811c49dc15a78945fedcd77d2bf6538d47b817e7caa2b46fa2

In [31]:
join_results = pd.read_parquet('../src/join_results_2col_MATE.parquet')
count_col_types(join_results, 'key_col').most_common()

[('Unnamed: 0', 80),
 ('Citance Number', 6),
 ('Citation Text', 3),
 ('id', 2),
 ('date', 2),
 ('Line Number', 1),
 ('SELECTION_ID', 1),
 ('Message_id', 1),
 ('Name', 1),
 ('mag', 1),
 ('transcript_index', 1),
 ('answerid', 1)]

In [34]:
groupby_results = pd.read_parquet('../src/gb_results_2col_MATE.parquet')

def count_gb_col_types(df, arg_string):
    counter = Counter()

    for ix, row in df.iterrows():
        for col in row['args'][arg_string]:
            counter[col]= counter.get(col, 0) + 1
                    
    return counter

count_gb_col_types(groupby_results, 'colset').most_common()

[('URL', 23),
 ('Comments', 23),
 ('Title', 22),
 ('Points', 20),
 ('Type', 19),
 ('act_tag', 5),
 ('Story', 4),
 ('speaker', 3),
 ('Namespace', 3),
 ('ce', 2),
 ('Unnamed: 0', 2),
 ('Table 5', 2),
 ('cbm', 2),
 ('genus', 2),
 ('File', 2),
 ('noc', 2),
 ('EVENT_NAME', 2),
 ('MORNINGTRADEDVOL', 2),
 ('topic_64', 1),
 ('sal5', 1),
 ('cbo', 1),
 ('End Date', 1),
 ('Location', 1),
 ('ObjectOriented Programming and Data Structures II', 1),
 ('predicted score', 1),
 ('LayerType', 1),
 ('RealTime(ms)', 1),
 ('HDPE', 1),
 ('O', 1),
 ('99%', 1),
 ('Win (1 of 2)', 1),
 ('ltable_year', 1),
 ('rtable_year', 1),
 ('in_iM', 1),
 ('category', 1),
 ('swda_filename', 1),
 ('Cause', 1),
 ('wmc', 1),
 ('max_cc', 1),
 ('Forum', 1),
 ('Author', 1),
 ('status', 1),
 ('ts', 1),
 ('subutterance_index', 1),
 ('Race Discipline', 1),
 ('Race Category', 1),
 ('topic_32', 1),
 ('0.1', 1),
 ('ca', 1),
 ('96%', 1),
 ('No win (2 of 5)', 1),
 ('LOC', 1),
 ('OREB_PCT', 1),
 ('DREB_PCT', 1),
 ('authors', 1),
 ('base_cod

In [8]:
# Find token length distribution for the queries
import pandas as pd
import pyarrow as pa


def find_token_length(query_df, df_label='dst_label', query_col='colset'):
    token_lengths = []
    for ix, row in query_df.iterrows():
        dst_col = row['args'][query_col]
        dst_df = pd.read_parquet(row[df_label])
        if type(dst_col) == set:
            for col in dst_col:
                token_lengths.extend(dst_df[col].apply(lambda x: len(str(x))).values.tolist())
        else:
            token_lengths.extend(dst_df[dst_col].apply(lambda x: len(str(x))).values.tolist())
    return token_lengths
    
groupby_results = pd.read_parquet('../src/gb_results_2col_MATE.parquet')
gb_tokens = find_token_length(groupby_results)
pd.DataFrame(gb_tokens).describe()

,0
count,200.000000
mean,521.795000
std,468.990312
min,102.000000
25%,192.000000
50%,319.000000
75%,698.000000
max,2658.000000


In [6]:
groupby_results

,src_labels,operation,args,dst_label,colset_size,results
200,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['Type', 'URL'],...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[07-08_1419.parquet, 08-09_1760.parquet, 07-08..."
201,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['topic_64', 'sa...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[tokens_1838.parquet, tokens_37.parquet, token..."
202,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['Comments', 'UR...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[agreement_across_PP.parquet, Delaware.parquet..."
203,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['Type', 'Points...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[23-24_714.parquet, 07-08_1419.parquet, 04-05_..."
204,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['cbo', 'ce'], '...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[contradicting_6_nouns.parquet, agreement_acro..."
...,...,...,...,...,...,...
295,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['n_consonants',...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[contradicting_6_nouns.parquet, agreement_acro..."
296,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['Comments', 'Ti...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[agreement_across_PP.parquet, PL2331LAGM5SHJ.p..."
297,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['Story', 'Comme...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[Delaware.parquet, PL2331LAGM5SHJ.parquet, PL2..."
298,[/tank/local/suhail/data/relic-datalake/gittab...,groupby,"{'agg_func': 'sum', 'colset': ['EVENT_NAME', '...",/tank/local/suhail/data/relic-datalake/gittabl...,2.0,"[dwbfpricesusaplace21102015.parquet, dwbfprice..."


In [20]:
def find_src_token_length(query_df, df_label='src_df', query_col='colset'):
    token_lengths = []
    for ix, row in query_df.iterrows():
        dst_col = row['args'][query_col]
        dst_df = pd.read_parquet(row[df_label][0])
        #print(dst_df.columns, dst_col)
        if type(dst_col) == set:
            for col in dst_col:
                token_lengths.extend(dst_df[col].apply(lambda x: len(str(x))).values.tolist())
        else:
            token_lengths.extend(dst_df[dst_col].apply(lambda x: len(str(x))).values.tolist())
    return token_lengths

pivot_results = pd.read_parquet('../src/pivot_results_2col_MATE.parquet')
pivot_tokens = find_src_token_length(pivot_results, df_label='src_labels', query_col='index_col')
pd.DataFrame(pivot_tokens).describe()

,0
count,6826.000000
mean,38.551568
std,139.383505
min,1.000000
25%,4.000000
50%,16.000000
75%,25.000000
max,3693.000000


In [1]:
pivot_results = pd.read_parquet('../src/pivot_results_2col_MATE.parquet')
pivot_tokens = find_src_token_length(pivot_results, df_label='src_labels', query_col='index_col')
pd.DataFrame(pivot_tokens).describe()

NameError: name 'pd' is not defined

In [21]:
join_results = pd.read_parquet('../src/join_results_2col_MATE.parquet')
join_tokens = find_token_length(join_results, query_col='key_col')
pd.DataFrame(join_tokens).describe()


,0
count,4072.000000
mean,7.533644
std,25.759938
min,1.000000
25%,2.000000
50%,2.000000
75%,10.000000
max,567.000000
